# Build sudoku game board and elements

In [1]:

# cross function
def cross(A, B):
    """Cross product of elements in A and elements in B.
    Args:
        A(str): string of rows e.g. 'ABCDEFGHI'
        B(str): string of columns e.g. '123456789'
        
    Returns:
        list of cross products between A and B 
        e.g. ['A1', 'A2', ... 'I9']
    """
    return [a+b for a in A for b in B]

# game board elements: rows and cols and all boxes
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = cross(rows, cols)

# units: completed rows, cols, squares, diagonals
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
row_groups = [rows[i:i+3] for i in range(0,7,3)]
column_groups = [cols[i:i+3] for i in range(0,7,3)]
square_units = [cross(rg, cg) for rg in row_groups for cg in column_groups]
diag_units = [[r+c for r, c in zip(rows, cols)], [r+c for r, c in zip(rows, reversed(cols))]]

# unit list, units of a box, peers of a box
#unitlist = row_units + column_units + square_units + diag_units
unitlist = row_units + column_units + square_units + diag_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

# misc
double_digits = [a+b for a in '123456789' for b in '123456789' if a<b]

In [11]:
peers['H8']

{'A1',
 'A8',
 'B2',
 'B8',
 'C3',
 'C8',
 'D4',
 'D8',
 'E5',
 'E8',
 'F6',
 'F8',
 'G7',
 'G8',
 'G9',
 'H1',
 'H2',
 'H3',
 'H4',
 'H5',
 'H6',
 'H7',
 'H9',
 'I7',
 'I8',
 'I9'}

# Display game

In [2]:
# make and display sudoku dict

def grid_values(grid):
    return {key:'123456789' if grid[index]=='.' else grid[index] 
            for index, key in enumerate(boxes)}

def grid_values_for_dot_display(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    return {key:grid[index] for index, key in enumerate(boxes)}

def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

# game state is input by string A1 A2 A3 ... A9 B1 ... B9 .. I9 
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

# game state in dict form
grid_dict = grid_values(grid)

# print game state
display(grid_values_for_dot_display(grid))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


# Eliminate impossible game actions

In [14]:
# flatten python list
def ravel(nested_list):
    """Similar to np.ravel
    Args:
        nested_list(list)
        
    Returns:
        flatten(list): flattened list
    """
    if isinstance(nested_list,list):
        flatten = []
        for units_ in nested_list:
            if isinstance(units_,list):
                flatten.extend(units_)
            else:
                flatten.append(units_)    
        if True in [isinstance(flatten[i], list) for i in range(len(flatten))]:
            return ravel(flatten)
        return flatten
    return nested_list

def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values recursively.
    """
#     for current_box in boxes:
#         current_box_len = len(values[current_box])
#         if current_box_len>1:
#             #boxes_to_scan = ravel([unit for unit in unitlist if current_box in unit])
#             #boxes_to_scan = set(boxes_to_scan) - set([current_box])
#             #boxes_to_scan = [b for b in boxes_to_scan if len(values[b])==1]
#             boxes_to_scan = [b for b in peers[current_box] if len(values[b])==1]
#             values_to_kill = [values[i] for i in boxes_to_scan]
#             new_values = ''.join(sorted(set(values[current_box]) - set(values_to_kill)))
# #             new_values = ''.join(set(values[current_box]) - set(values_to_kill))
              
#             # update Sudoku dict values if any current box values are eliminated
#             if len(new_values)<current_box_len:
#                 values[current_box] = new_values
#                 return eliminate(values)
#     return values

    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

temp = grid_values(grid)
temp = eliminate(temp)
display(temp)

   4     4578    3   |  149     2     147  |   6     5789    5   
   9     2467   247  |   3      47     5   |   78     8      1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  1345   1345    8   |         3456    2   |   9    134567 34567 
   7    123459  249  |  1459   369     14  |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  1234  12347        |   6     478     9   |   5    12478   247  
   8      16     47  |   2     457     3   |   17    467     9   
   6    24679    5   |   4      1      47  |   3    24678   2467 


# Only Choice

In [15]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    
    Args:
        values: Sudoku in dictionary form.
        
    Returns:
        Resulting Sudoku in dictionary form after applying only_choice recursively.
    """
    
    for unit in unitlist:
        for choice in '123456789':
            pos_for_a_choice = [box for box in unit if choice in values[box]]
            if len(pos_for_a_choice) == 1 and len(values[pos_for_a_choice[0]]) != 1:
                values[pos_for_a_choice[0]] = choice
                return only_choice(values)
    return values

grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

temp = grid_values(grid)
temp = only_choice(temp)
display(temp)

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

# Naked twins

In [16]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    for unit in unitlist:
        for choice in double_digits:
            twins_position = [box for box in unit if choice == values[box]]
            if len(twins_position) == 2:
                boxes_to_eliminate = [box for box in unit if any(char in values[box] for char in choice) and box not in twins_position]
                if len(boxes_to_eliminate)>0:
                    for box in boxes_to_eliminate:
                        values[box] = values[box].replace(choice[0],'').replace(choice[1],'')
                    return naked_twins(values)
    return values

grid3 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    
temp = grid_values(grid3)
temp = naked_twins(temp)

display(temp)

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

# DFS

In [17]:
def reduce_puzzle(values):
    """Iterate eliminate() and only_choice(). 
    
    If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    
    Args:
        values: Sudoku in dictionary form.
        
    Returns:
        case1: Return Sudoku in dictionary form after iterating eliminate and only_choice.
        case2: Return False if no solution (No available choice for any one of the boxes).
    """
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        
        # stalled = true if no progress is made. Then values are returned.
        stalled = solved_values_before == solved_values_after
        
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def search(values):
    """Using depth-first search and propagation, create a search tree and solve the sudoku.
    
    Use reduce_puzzle to apply constraint prop first. If no progress, DFS is applied
    
    Args:
        values: Sudoku in dictionary form.
        
    Returns:
        case1: Return solved Sudoku dict if solvable.
        case2: Return False if no solution exists.
    """
    # Reduce the puzzle first
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier. No solution for the given puzzle
    if all(len(values[s]) == 1 for s in boxes):
        return values ## Solved!
    
    # For any irreduciable puzzle, choose one least entropic unfilled squares to do DFS
    n,box = min((len(values[box]), box) for box in boxes if len(values[box]) > 1)
    for value in values[box]:
        new_sudoku = values.copy() ## use copy for not corrupting the true game state
        new_sudoku[box] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt
        
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
   
temp = grid_values(grid2)
temp = search(temp)
if temp:
    display(temp)
else:
    print('No solution')

No solution


In [18]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    return search(grid_values(grid))


grid4 = '9.1....8.8.5.7..4.2.4....6...7......5..............83.3..6......9................'
grid5 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

try:
    display(solve(grid4))
except:
    print('No solution')
print()


9 6 1 |3 5 4 |2 8 7 
8 3 5 |2 7 6 |1 4 9 
2 7 4 |8 9 1 |3 6 5 
------+------+------
4 2 7 |1 3 8 |9 5 6 
5 8 3 |9 6 2 |7 1 4 
6 1 9 |5 4 7 |8 3 2 
------+------+------
3 4 2 |6 8 9 |5 7 1 
7 9 8 |4 1 5 |6 2 3 
1 5 6 |7 2 3 |4 9 8 



In [ ]:
solved_diag_sudoku = {'G7': '8', 'G6': '9', 'G5': '7', 'G4': '3', 'G3': '2', 'G2': '4', 'G1': '6', 'G9': '5',
                          'G8': '1', 'C9': '6', 'C8': '7', 'C3': '1', 'C2': '9', 'C1': '4', 'C7': '5', 'C6': '3',
                          'C5': '2', 'C4': '8', 'E5': '9', 'E4': '1', 'F1': '1', 'F2': '2', 'F3': '9', 'F4': '6',
                          'F5': '5', 'F6': '7', 'F7': '4', 'F8': '3', 'F9': '8', 'B4': '7', 'B5': '1', 'B6': '6',
                          'B7': '2', 'B1': '8', 'B2': '5', 'B3': '3', 'B8': '4', 'B9': '9', 'I9': '3', 'I8': '2',
                          'I1': '7', 'I3': '8', 'I2': '1', 'I5': '6', 'I4': '5', 'I7': '9', 'I6': '4', 'A1': '2',
                          'A3': '7', 'A2': '6', 'E9': '7', 'A4': '9', 'A7': '3', 'A6': '5', 'A9': '1', 'A8': '8',
                          'E7': '6', 'E6': '2', 'E1': '3', 'E3': '4', 'E2': '8', 'E8': '5', 'A5': '4', 'H8': '6',
                          'H9': '4', 'H2': '3', 'H3': '5', 'H1': '9', 'H6': '1', 'H7': '7', 'H4': '2', 'H5': '8',
                          'D8': '9', 'D9': '2', 'D6': '8', 'D7': '1', 'D4': '4', 'D5': '3', 'D2': '7', 'D3': '6',
                          'D1': '5'}

display(solved_diag_sudoku)

In [ ]:
    
before_naked_twins_1 = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
                            'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
                            'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
                            'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
                            'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
                            'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
                            'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
                            'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
                            'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
                            'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
                            'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}

display(before_naked_twins_1)

print()
print()

possible_solutions_1 = [
        {'G7': '6', 'G6': '3', 'G5': '2', 'G4': '9', 'G3': '1', 'G2': '8', 'G1': '7', 'G9': '5', 'G8': '4', 'C9': '1',
         'C8': '5', 'C3': '8', 'C2': '237', 'C1': '23', 'C7': '9', 'C6': '6', 'C5': '37', 'A4': '2357', 'A9': '8',
         'A8': '6', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'F6': '125', 'F7': '35', 'F8': '9',
         'F9': '7', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'C4': '4',
         'B8': '3', 'B9': '4', 'I9': '9', 'I8': '7', 'I1': '23', 'I3': '23', 'I2': '6', 'I5': '5', 'I4': '8', 'I7': '1',
         'I6': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'E8': '1', 'A7': '27', 'A6': '257', 'E5': '347',
         'E4': '6', 'E7': '345', 'E6': '579', 'E1': '8', 'E3': '79', 'E2': '37', 'H8': '2', 'H9': '3', 'H2': '9',
         'H3': '5', 'H1': '4', 'H6': '17', 'H7': '8', 'H4': '17', 'H5': '6', 'D8': '8', 'D9': '6', 'D6': '279',
         'D7': '34', 'D4': '237', 'D5': '347', 'D2': '1', 'D3': '79', 'D1': '5'},
        {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8', 'H5': '6', 'F9': '7',
         'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8', 'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23',
         'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5', 'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9',
         'A4': '2357', 'A7': '27', 'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
         'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2', 'F6': '125',
         'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '79', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235',
         'F5': '8', 'E2': '3', 'F7': '35', 'F8': '9', 'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17',
         'D3': '79', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6',
         'D6': '279', 'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}
        ]


temp = naked_twins(before_naked_twins_1)
display(temp)




In [ ]:

for i in possible_solutions_1:
    display(i)
    print()

In [19]:
diag_units

[['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9'],
 ['A9', 'B8', 'C7', 'D6', 'E5', 'F4', 'G3', 'H2', 'I1']]